# Overview

(This notebook is for use with SageMath, as in particular it makes heavy use of functions for toric geometry.)

This notebook is about two things:

First, we want to realise the (2,4) Calabi-Yau, i.e. CICY#7887 as a toric complete intersection corresponding to the Cox ring representation given in 1305.0537. We want to show that this is indeed the (2,4) (and then subsequently we will attempt to use it to derive the generating function for the Cox ring of the (2,4)).

Second, we want to take the fact that we know the generating function for the h1 values on the (2,4), and try to use this to build a 'h1-space' whose h0 values will give the h1 values on the (2,4).

# Functions

In [8]:
# The divisor 'div' defines the hypersurface inside the toric variety given by the fan 'fan'
def bertinis_theorem_holds_for_div(fan,div):
    """
    Checks whether Bertini's theorem holds for a given hypersurface inside a given toric ambient space

    :param fan: the fan which defines the toric ambient space
    :param div: the divisor which defines the hypersurface
    :return: a Boolean value
    """ 
    
    tv = ToricVariety(fan)
    
    num_coords = len(fan(1))
    
    zero_coord_set_list = []
    for top_dim_cone in fan(fan.dim()): #collect all the largest sets of coordinates that can vanish together
    
        ray_list = [ray.list() for ray in top_dim_cone.rays()]
        
        fan_coord_list = []
        i=0
        for c in fan(1):
            if c.rays().matrix().list() in ray_list:
                fan_coord_list.append(i)
            i += 1
            
        zero_coord_set_list = zero_coord_set_list + [fan_coord_list]
    
    monoms = div.sections_monomials()
    monoms_degrees = [monom.degrees() for monom in monoms] #now collect all monomials

    for monom_degrees in monoms_degrees:
        if len(monom_degrees) != num_coords:
            print("Error: Degrees aren't being handled correctly")
            return None
    
    for zero_coord_set in zero_coord_set_list: #go through all the sets of vanishing coordinates
        monoms_all_zero = True
        for monom_degrees in monoms_degrees: #for each go through all sets of monomials
            monom_zero = False
            for i in zero_coord_set:
                if monom_degrees[i] != 0: #check if a coordinate being set to zero appears in this monomomial
                    monom_zero = True
            if monom_zero == False:
                monoms_all_zero = False
                break
        if monoms_all_zero == True:
            #print "Found somewhere where all monomials vanish"
            return False
            break
            
    return True

# The alternative description of the (2,4)

We know from Ottem's work 1305.0537 the vertices of the polytope for an alternative description of the (2,4). Here we investigate its properties.

In [ ]:
vert = [[1,1,0,0,0,-1],[0,0,0,0,0,1],[-4,-4,-1,-1,-1,0],[0,0,0,0,1,0],
        [0,0,0,1,0,0],[0,0,1,0,0,0],[0,1,0,0,0,0],[1,0,0,0,0,0]]

LP = LatticePolytope(vert)
P = Polyhedron(vertices=vert)
m = P.dim()
origin = [0 for i in range(m)]

print("Is it reflexive? " + str(P.is_reflexive()))
print("Does it have the IP property? " + str(P.has_IP_property()) + "\n")

print("Consider the fan(s) defined by these rays")

Pint = ((0, 0, 0, 0, 0, 0),
 (1, 1, 0, 0, 0, -1),
 (0, 0, 0, 0, 0, 1),
 (-4, -4, -1, -1, -1, 0),
 (0, 0, 0, 0, 1, 0),
 (0, 0, 0, 1, 0, 0),
 (0, 0, 1, 0, 0, 0),
 (0, 1, 0, 0, 0, 0),
 (1, 0, 0, 0, 0, 0))
origin = [0 for i in range(m)]
PC = PointConfiguration(Pint).restrict_to_fine_triangulations(fine=True).restrict_to_star_triangulations(origin)   
trs = PC.triangulations()  

print("Looping over all triangulations:")

for tr in trs:
    
    print("\nChecking a triangulation ...")
    
    F = Fan(tr,Pint)
    T = ToricVariety(F, coordinate_names="x0 x1 y0 y1 y2 y3 z0 z1 e")

    print("Is it smooth? " + str(T.is_smooth()))
    
    if T.is_smooth():
        
        print(F.rays()
    
        print("Triangulation: " + str(tr))
        print(T.Stanley_Reisner_ideal())
    
    else:
        
        singularityCodimensionFound = False
        
        for i in range(m):
            
            if singularityCodimensionFound == True:
            
                break
            
            coneList = F.cones(i)
            
            for cone in coneList:
                
                if cone.is_smooth() == False:
                    
                    singularityCodimensionFound = True
                    print("Singularities occur at codimension " + str(i))
                    break

We learn from the above that the singularities occur at codimension 4, so they will generically miss the CY. Let's look at including all the interior points of the polytope. This should fill in the required rays to make the fans smooth.

In [ ]:
# Pint = P.integral_points()
# The above introduces a single extra ray, but it also permutes the list around, 
# so I'll just hardcode the list in instead in a way which doesn't permute the old rays
Pint = ((0, 0, 0, 0, 0, 0),
 (1, 1, 0, 0, 0, -1),
 (0, 0, 0, 0, 0, 1),
 (-4, -4, -1, -1, -1, 0),
 (0, 0, 0, 0, 1, 0),
 (0, 0, 0, 1, 0, 0),
 (0, 0, 1, 0, 0, 0),
 (0, 1, 0, 0, 0, 0),
 (1, 0, 0, 0, 0, 0),
 (-1, -1, 0, 0, 0, 0))
PC = PointConfiguration(Pint).restrict_to_fine_triangulations(fine=True).restrict_to_star_triangulations(origin)   
trs = PC.triangulations()  

print("\nLooping over all triangulations:")

for tr in trs:
    
    print("\nChecking a triangulation ...")
    
    F = Fan(tr,Pint)
    T = ToricVariety(F)
    
    print("Is it smooth? " + str(T.is_smooth()))
    
    if T.is_smooth():
        
        print("")
        print(F.rays())
    
        print("Triangulation: " + str(tr))
        print(T.Stanley_Reisner_ideal())

We see that we can resolve the ambient space to give a smooth one, which would be nicer to work with. But does this blow-up resolution miss the complete intersection, so that we don't affect the Calabi-Yau? Let's check this.

In [ ]:
CR = T.cohomology_ring()
C = [CR(c) for c in F(1)]
complInt = (4*C[2]+C[-1])*(4*C[2]+C[-1])*(4*C[2]+C[-1])

# Use the below to see what a good basis is, to see what intersections we need to check
# print("cohomology ring = " + str(C))
# print("basis = " + str(T.cohomology_basis(1)))

# We want to check whether the blow-up divisor class C[-1] intersects the complete intersection
# We do this by checking all the top intersection numbers
# The rest of the basis is spanned by C[0] and C[2]
print("\nChecking top intersections ... " + str(T.integrate(C[0]*C[0]*C[-1]*complInt)) + " ... " + str(T.integrate(C[0]*C[2]*C[-1]*complInt)) + " ... " + str(T.integrate(C[2]*C[2]*C[-1]*complInt))

We see from the above that the blow-up resolution misses the complete intersection. Hence we can straightforwardly use this complete intersection description inside this smooth ambient space.

The final check to establish smoothness of the Calabi-Yau itself is Bertini's theorem, to see if the sections themselves (of the various individual defining line bundles) are generically smooth.

In [ ]:
# Note we know which divisor indices correspond to which coordinates because we hardcoded the ordering
# More generally we would first have to check what ordering the fan uses

D_x = T.divisor(0)
D_y = T.divisor(2)
D_z = T.divisor(6)
D_e = T.divisor(8)
D_0 = 4*D_y + D_e

print("Checking ... " + str(bertinis_theorem_holds_for_div(F,D_0)))

# The 'h1-space'

Having satisfied ourselves that the alternative complete intersection description above indeed gives a good description of the (2,4), we now turn to the construction and analysis of the associated 'h1-space'.

We generate the vertices of a 'h1-polytope' by analogy with how the h1 values are obtained from h0.

In [ ]:
vert = [[-1,-1,0,0,0,-1],[0,0,0,0,0,1],[-4,-4,-1,-1,-1,0],[0,0,0,0,1,0],
        [0,0,0,1,0,0],[0,0,1,0,0,0],[0,1,0,0,0,0],[1,0,0,0,0,0]]

LP = LatticePolytope(vert)
P = Polyhedron(vertices=vert)
m = P.dim()
origin = [0 for i in range(m)]

print("Is it reflexive? " + str(P.is_reflexive()))
print("Does it have the IP property? " + str(P.has_IP_property()) + "\n")

print("Consider the fan(s) defined by these rays")

Pint = ((0, 0, 0, 0, 0, 0),
 (-1, -1, 0, 0, 0, -1),
 (0, 0, 0, 0, 0, 1),
 (-4, -4, -1, -1, -1, 0),
 (0, 0, 0, 0, 1, 0),
 (0, 0, 0, 1, 0, 0),
 (0, 0, 1, 0, 0, 0),
 (0, 1, 0, 0, 0, 0),
 (1, 0, 0, 0, 0, 0))
origin = [0 for i in range(m)]
PC = PointConfiguration(Pint).restrict_to_fine_triangulations(fine=True).restrict_to_star_triangulations(origin)   
trs = PC.triangulations()  

print("Looping over all triangulations:")

for tr in trs:
    
    print("\nChecking a triangulation ...")
    
    F = Fan(tr,Pint)
    T = ToricVariety(F, coordinate_names="x0 x1 y0 y1 y2 y3 z0 z1 e")

    print("Is it smooth? " + str(T.is_smooth()))
    
    if T.is_smooth():
        
        print(F.rays()
    
        print("Triangulation: " + str(tr))
        print(T.Stanley_Reisner_ideal())
    
    else:
        
        singularityCodimensionFound = False
        
        for i in range(m):
            
            if singularityCodimensionFound == True:
            
                break
            
            coneList = F.cones(i)
            
            for cone in coneList:
                
                if cone.is_smooth() == False:
                    
                    singularityCodimensionFound = True
                    print("Singularities occur at codimension " + str(i))
                    break

As for the (2,4) itself, we learn from the above that the singularities occur at codimension 4, so they will generically miss the CY. Hence we again look at including all the interior points of the polytope. This should fill in the required rays to make the fans smooth.

In [ ]:
# Pint = P.integral_points()
# The above introduces a single extra ray, but it also permutes the list around, 
# so I'll just hardcode the list in instead in a way which doesn't permute the old rays
Pint = ((0, 0, 0, 0, 0, 0),
 (-1, -1, 0, 0, 0, -1),
 (0, 0, 0, 0, 0, 1),
 (-4, -4, -1, -1, -1, 0),
 (0, 0, 0, 0, 1, 0),
 (0, 0, 0, 1, 0, 0),
 (0, 0, 1, 0, 0, 0),
 (0, 1, 0, 0, 0, 0),
 (1, 0, 0, 0, 0, 0),
 (-1, -1, 0, 0, 0, 0))
PC = PointConfiguration(Pint).restrict_to_fine_triangulations(fine=True).restrict_to_star_triangulations(origin)   
trs = PC.triangulations()  

print("\nLooping over all triangulations:")

for tr in trs:
    
    print("\nChecking a triangulation ...")
    
    F = Fan(tr,Pint)
    T = ToricVariety(F)
    
    print("Is it smooth? " + str(T.is_smooth()))
    
    if T.is_smooth():
        
        print("")
        print(F.rays())
    
        print("Triangulation: " + str(tr))
        print(T.Stanley_Reisner_ideal())

Again as for the (2,4) itself, the blow-up resolution misses the complete intersection.

But what happens when we check Bertini's theorem?

In [ ]:
D_x = T.divisor(0)
D_y = T.divisor(2)
D_z = T.divisor(6)
D_e = T.divisor(8)
D_0 = 4*D_y + D_e
    
print()"Checking ... " + str(bertinis_theorem_holds_for_div(F,D_0)))

We see that Bertini's theorem fails for this 'h1-space'. Since this is only a sufficient, rather than a necessary, condition for smoothness of the complete intersection, this isn't conclusive. However, it is very suggestive.

Unfortunately, this is the only straightforward check one can perform. A definitive answer could be obtained by building a generic (set of) sections, and seeing if they are smooth or not. Here we can use T.subscheme(...).is_smooth(). However, using all of the monomials of the relevant line bundles creates a section whose smoothness is computationally prohibitive to check. 